<a href="https://colab.research.google.com/github/zbooster/Landmark-Image-Classification/blob/main/landmark_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 준비


## Google 드라이브에서 Resize된 데이터 가져오기
gdown과 unzip을 이용하여 압축을 풀어서 나의 colab 폴더에 둔다

In [2]:
!gdown 1_yGDePPvjlsTe4qi3X6LqY2DPEB97Itq
!unzip -qq '/content/resizeds_0.1x.zip' -d './datasets'

Downloading...
From: https://drive.google.com/uc?id=1_yGDePPvjlsTe4qi3X6LqY2DPEB97Itq
To: /content/resizeds_0.1x.zip
100% 837M/837M [00:03<00:00, 211MB/s]


# 학습준비

In [3]:
import torch
import os

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
BATCH_SIZE = 256
EPOCH = 30

In [4]:
import torchvision.transforms as transforms

data_transforms = {
    'Training': transforms.Compose([transforms.Resize([64, 64]),
        transforms.RandomHorizontalFlip(), transforms.RandomVerticalFlip(), 
        transforms.RandomCrop(52), transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ]),
    'Validation':  transforms.Compose([transforms.Resize([64, 64]),
        transforms.RandomCrop(52), transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ])
}

- 데이터 분류가 전처리(Resize) 과정에서 완료되었으므로 압축해제한 폴더를 그대로 사용한다.

In [5]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

data_dir = '/content/datasets/resizeds'
image_datasets = {x: ImageFolder(root=os.path.join(data_dir, x),
                                 transform=data_transforms[x]) for x in data_transforms.keys()}
dataloaders = {x: DataLoader(image_datasets[x],
                             batch_size=BATCH_SIZE,
                             shuffle=True,
                             num_workers=2) for x in data_transforms.keys()}     
dataset_sizes = {x: len(image_datasets[x]) for x in data_transforms.keys()}      
class_name = image_datasets['Training'].classes

- pretrained 옵션이 deprecated되어 가이드에 나와있는 다른 방법으로 weights를 설정
[참조1](https://pytorch.org/vision/stable/models.html)

In [6]:
from torchvision.models import resnet50, ResNet50_Weights
import torch.nn as nn
import torch.optim as optim

# resnet = models.resnet50(pretrained=True) # deprecated
# New weights with accuracy 80.858%
resnet = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2) 
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 84)
resnet = resnet.to(DEVICE)

critetion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=0.001)

from torch.optim import lr_scheduler
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [7]:
for idx, child in enumerate(resnet.children()):
  if idx < 6:
    for param in child.parameters():
      param.requires_grad = False

In [8]:
import time
import copy

def train_resnet(model, criterion, optimizer, scheduler, num_epochs=25):

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
  

  for epoch in range(1, num_epochs + 1):
    since = time.time()
    for phase in data_transforms.keys():
      if phase == 'Training':
        model.train()
      else:
        model.eval()

      running_loss = 0.0
      running_corrects = 0

      for inputs, labels in dataloaders[phase]:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        optimizer.zero_grad()

        with torch.set_grad_enabled(phase == 'Training'):
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels)

          if phase == 'Training':
            loss.backward()
            optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

      if phase == 'Training':
        scheduler.step()

      epoch_loss = running_loss/dataset_sizes[phase]
      epoch_acc = running_corrects.double()/dataset_sizes[phase]

      print('{} Loss: {:.4f}, Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

      if phase == 'Validation' and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
  print('Best Validation Acc: {:4f}'.format(best_acc))

  model.load_state_dict(best_model_wts)

  return model

# 학습하기

In [9]:
model_resnet50 = train_resnet(resnet, critetion, optimizer_ft,
                              exp_lr_scheduler, num_epochs=EPOCH)

torch.save(model_resnet50, 'resnet50.pt')

Training Loss: 1.4826, Acc: 0.6617
Validation Loss: 0.4647, Acc: 0.8723
Completed in 0m 46s
Training Loss: 0.1299, Acc: 0.9626
Validation Loss: 0.3863, Acc: 0.9036
Completed in 0m 40s
Training Loss: 0.0706, Acc: 0.9788
Validation Loss: 0.2897, Acc: 0.9235
Completed in 0m 41s
Training Loss: 0.0468, Acc: 0.9861
Validation Loss: 0.2476, Acc: 0.9302
Completed in 0m 40s
Training Loss: 0.0443, Acc: 0.9877
Validation Loss: 0.1662, Acc: 0.9548
Completed in 0m 39s
Training Loss: 0.0344, Acc: 0.9892
Validation Loss: 0.1861, Acc: 0.9501
Completed in 0m 40s
Training Loss: 0.0280, Acc: 0.9912
Validation Loss: 0.1826, Acc: 0.9448
Completed in 0m 41s
Training Loss: 0.0166, Acc: 0.9941
Validation Loss: 0.1141, Acc: 0.9654
Completed in 0m 40s
Training Loss: 0.0074, Acc: 0.9982
Validation Loss: 0.1325, Acc: 0.9641
Completed in 0m 40s
Training Loss: 0.0071, Acc: 0.9980
Validation Loss: 0.1258, Acc: 0.9634
Completed in 0m 41s
Training Loss: 0.0085, Acc: 0.9973
Validation Loss: 0.1137, Acc: 0.9614
Complete

In [10]:
!cp '/content/resnet50.pt' '/content/drive/MyDrive/DeepLearning_Datas/resnet50_2022_08_02_17_40_00.pt'